In [1]:
import pandas as pd
import numpy as np

'''
Author: Enrico Ceccolini
    remove from the original data jobs with:
    - missing end_time
    - missing shedule informations (in which nodes was computed)
    - wrong schedule informations (scheduled on unexisting node)
    
    compute the missing jobs' requirement by looking to the schedule data (dummy alg. version)
    (a lot of jobs have g)
    
    output the csv of the cleaned data
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/"
infile_jobs_original = datadir + "jobs"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

# settings
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')

interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')

#interval_comment = "WholeData"
#start_time_whole = pd.to_datetime('2014-03-24')
#end_time_whole = pd.to_datetime('2015-08-12')

outfile_jobs_cleaned = datadir + "CPUs/" + interval_comment + "/jobs_cleaned"

# read dataset
jobs_original = pd.read_csv(infile_jobs_original + ".csv", index_col=0) # Read a comma-separated values (csv) file into DataFrame
jobs_original.shape

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(405771, 15)

In [13]:
jobs_original.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code
job_id,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN


In [14]:
# remove the jobs with missing end-time
jobs_completed = jobs_original[jobs_original.end_time != '0000-00-00 00:00:00']
print("completed_jobs_data contains {} records".format(jobs_completed.shape[0]))

completed_jobs_data contains 404871 records


recover the missing required resources

In [15]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

# remove jobs runned on the node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]

## remove jobs with the same id that runned on the same node
#jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
#print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


In [16]:
### note the multiple sheduling on the same node, it's legit
jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['job_id_string'] == '502893.node129']

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
3114,7,502893.node129,2,0,0,3145728
3115,7,502893.node129,2,0,0,3145728
3116,8,502893.node129,2,0,0,3145728
3117,8,502893.node129,2,0,0,3145728
3118,8,502893.node129,2,0,0,3145728
3119,8,502893.node129,2,0,0,3145728
3120,8,502893.node129,2,0,0,3145728
3121,9,502893.node129,2,0,0,3145728


In [17]:
# no ncpus greter than 16
jobs_to_nodes_whole_data[['job_id_string', 'ncpus']].groupby(['job_id_string']).mean()

,ncpus
job_id_string,
1006691.node129,16
1006692.node129,1
1006693.node129,16
1006694.node129,16
1006695.node129,8
1006696.node129,16
1006697.node129,16
1006698.node129,1
1006699.node129,1


In [18]:
# remove jobs with no scheduling informations
jobs_completed = jobs_completed[jobs_completed['job_id_string'].isin(jobs_to_nodes_whole_data['job_id_string'])]
print("completed_jobs_data contains {} records".format(jobs_completed.shape[0]))

completed_jobs_data contains 404792 records


In [19]:
jobs_to_nodes_merged = pd.merge(jobs_to_nodes_whole_data, jobs_completed, on='job_id_string')
print("jobs_to_nodes_merged contains {} records".format(jobs_to_nodes_merged.shape[0]))

jobs_to_nodes_merged contains 467691 records


In [20]:
# set the right number of nodes where is missing
jobs_zero_nodes = jobs_completed[jobs_completed['node_req'] == 0]
jobs_zero_nodes.shape

(326019, 15)

In [21]:
#jobs_completed[jobs_completed['node_req'] == 0]

In [22]:
i = 0
for index, row in jobs_completed.iterrows():
    if(i%1000 == 0):
        print("{}/{}".format(i, jobs_completed.shape[0]))
        jobs_completed.to_csv(outfile_jobs_cleaned + ".csv")
    if (row['node_req'] == 0):
        job_to_nodes = jobs_to_nodes_merged[jobs_to_nodes_merged['job_id_string'] == row['job_id_string']]
        nnodes = job_to_nodes.shape[0]
        ncpus = job_to_nodes['ncpus'].sum()
        ngpus = job_to_nodes['ngpus'].sum()
        nmics = job_to_nodes['nmics'].sum()
        mem_requested = job_to_nodes['mem_requested'].sum()

        #print(large_jobs.loc[index])
        #print(ncpus)
        jobs_completed.at[index, 'node_req'] = nnodes
        jobs_completed.at[index, 'cpu_req'] = ncpus
        jobs_completed.at[index, 'gpu_req'] = ngpus
        jobs_completed.at[index, 'mic_req'] = nmics
        jobs_completed.at[index, 'mem_req'] = mem_requested
    #jobs_to_nodes_merged = jobs_to_nodes_merged[jobs_to_nodes_merged['job_id_string'] != row['job_id_string']]
    i += 1
jobs_completed.to_csv(outfile_jobs_cleaned + ".csv")

0/404792
1000/404792
2000/404792
3000/404792
4000/404792
5000/404792
6000/404792
7000/404792
8000/404792
9000/404792
10000/404792
11000/404792
12000/404792
13000/404792
14000/404792
15000/404792
16000/404792
17000/404792
18000/404792
19000/404792
20000/404792
21000/404792
22000/404792
23000/404792
24000/404792
25000/404792
26000/404792
27000/404792
28000/404792
29000/404792
30000/404792
31000/404792
32000/404792
33000/404792
34000/404792
35000/404792
36000/404792
37000/404792
38000/404792
39000/404792
40000/404792
41000/404792
42000/404792
43000/404792
44000/404792
45000/404792
46000/404792
47000/404792
48000/404792
49000/404792
50000/404792
51000/404792
52000/404792
53000/404792
54000/404792
55000/404792
56000/404792
57000/404792
58000/404792
59000/404792
60000/404792
61000/404792
62000/404792
63000/404792
64000/404792
65000/404792
66000/404792
67000/404792
68000/404792
69000/404792
70000/404792
71000/404792
72000/404792
73000/404792
74000/404792
75000/404792
76000/404792
77000/404792

In [50]:
jobs_completed

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code,gpu_req,mic_req
job_id,,,,,,,,,,,,,,,,,
205722,1006691.node129,sub_Tio2_2x2x1_,parallel,2014-08-13 00:01:21,2014-08-13 00:03:14,2014-08-13 03:03:48,mpalummo@node129.eurora.cineca.it,1,16,0,NaN,NaN,NaN,COMPLETED,NaN,NaN,NaN
205723,1006692.node129,job022,debug,2014-08-13 00:29:54,2014-08-13 00:29:56,2014-08-13 01:00:18,ncreati0@node129.eurora.cineca.it,1,1,4194304,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205724,1006693.node129,rLdir10,parallel,2014-08-13 00:30:17,2014-08-13 00:30:19,2014-08-13 01:55:44,mdotti00@node129.eurora.cineca.it,1,16,14680064,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205725,1006694.node129,rHdir0,parallel,2014-08-13 00:30:31,2014-08-13 00:31:20,2014-08-13 00:51:30,mdotti00@node129.eurora.cineca.it,1,16,14680064,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205726,1006695.node129,runT2-15-800,np_longpar,2014-08-13 00:39:38,2014-08-13 00:39:39,2014-08-13 08:39:55,tlucchin@node129.eurora.cineca.it,2,16,16777216,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205727,1006696.node129,rlblast,parallel,2014-08-13 01:00:15,2014-08-13 01:10:07,2014-08-13 05:11:20,mdotti00@node129.eurora.cineca.it,1,16,14680064,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205728,1006697.node129,rHdir0,parallel,2014-08-13 01:00:19,2014-08-13 01:10:08,2014-08-13 01:30:18,mdotti00@node129.eurora.cineca.it,1,16,14680064,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205729,1006698.node129,job023,debug,2014-08-13 01:00:43,2014-08-13 01:00:44,2014-08-13 01:31:02,ncreati0@node129.eurora.cineca.it,1,1,4194304,NaN,NaN,NaN,COMPLETED,NaN,0.0,0.0
205730,1006699.node129,Gaox_xurlx,np_longpar,2014-08-13 01:31:02,2014-08-13 01:31:04,2014-08-13 01:52:06,gcocco00@node129.eurora.cineca.it,5,5,10485760,NaN,DEL,NaN,COMPLETED,NaN,0.0,0.0


In [23]:
jobs_completed = jobs_completed[jobs_completed['node_req'] != 0]
print(jobs_completed.shape[0])

404792


In [52]:
jobs_completed.to_csv(outfile_jobs_cleaned + ".csv")

### keep interval

In [2]:
#infile_jobs_cleaned = datadir + "jobs_cleaned"
jobs_cleaned = pd.read_csv(outfile_jobs_cleaned + ".csv", index_col=0) # Read a comma-separated values (csv) file into DataFrame
jobs_cleaned.shape

(404792, 17)

In [3]:
interval_jobs_cleaned = jobs_cleaned[pd.to_datetime(jobs_cleaned['start_time']) >= start_time]
interval_jobs_cleaned = interval_jobs_cleaned[pd.to_datetime(interval_jobs_cleaned['end_time']) <= end_time]
interval_jobs_cleaned.shape

(336787, 17)

In [4]:
interval_jobs_cleaned.to_csv(outfile_jobs_cleaned + ".csv")
#outfile_interval_jobs_cleaned = "/datasets/eurora_data/db_local/" + "CPUs/" + interval_comment + "/jobs_cleaned"
#interval_jobs_cleaned.to_csv(outfile_interval_jobs_cleaned + ".csv")
#outfile_interval_jobs_cleaned = "/datasets/eurora_data/db1/" + "CPUs/" + interval_comment + "/jobs_cleaned"
#interval_jobs_cleaned.to_csv(outfile_interval_jobs_cleaned + ".csv")